<a href="https://colab.research.google.com/github/Tatsuro0726/chemoinfomatics/blob/main/deepchem/Tutorial5_Createing_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Tutorial Part 5: Creating Models with TensorFlow and PyTorch
これまでのチュートリアルでは、DeepChemが提供する標準モデルを使用してきました。これは多くのアプリケーションには適していますが、遅かれ早かれ、自分で定義したアーキテクチャで全く新しいモデルを作成したいと思うでしょう。DeepChemは、TensorFlow（Keras）とPyTorchの両方との統合を提供しているので、これらのフレームワークのいずれかのモデルで使用することができます。

#### Setup

In [1]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3490  100  3490    0     0  15043      0 --:--:-- --:--:-- --:--:-- 15043


add /root/miniconda/lib/python3.6/site-packages to PYTHONPATH
python version: 3.6.9
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added conda-forge to channels
added omnia to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda



In [2]:
!pip install --pre deepchem

     |████████████████████████████████| 532kB 9.4MB/s 


#### Keras Model
Keras Modelは、DeepchemのModelクラスのサブクラス。
tensorflow.keras.Modelのラッパーとして動作する。

In [3]:
import deepchem as dc
import tensorflow as tf

# kerasのモデル構築
keras_model = tf.keras.Sequential([
                  tf.keras.layers.Dense(1000, activation='relu'),
                  tf.keras.layers.Dropout(rate=0.5),
                  tf.keras.layers.Dense(1)
])

# Deepchemでラップ
model = dc.models.KerasModel(keras_model,dc.models.losses.L2Loss())

In [5]:
# data読み込み
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='ECFP', splitter='random')
# datasetsの展開
train, valid, test = datasets

# モデルの学習
model.fit(train, nb_epoch=50)
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score) # モデルの評価指標
print('training set score: {}'.format(model.evaluate(train, [metric])))
print('test set score: {}'.format(model.evaluate(test, [metric])))

training set score: {'pearson_r2_score': 0.9793306697760502}
test set score: {'pearson_r2_score': 0.6481987386001775}


#### Torch Model
torch.nn.Moduleをラップ。

In [9]:
import torch

pytorch_model = torch.nn.Sequential(
                torch.nn.Linear(1024, 1000),
                torch.nn.ReLU(),
                torch.nn.Dropout(0.5),
                torch.nn.Linear(1000,1)
)
model = dc.models.TorchModel(pytorch_model, dc.models.losses.L2Loss())

model.fit(train, nb_epoch=50)
print('training set score: {}'.format(model.evaluate(train, [metric])))
print('test set score: {}'.format(model.evaluate(test, [metric])))

training set score: {'pearson_r2_score': 0.9790883000176276}
test set score: {'pearson_r2_score': 0.6357919894732449}


#### 損失を計算する
より発展した例の紹介。  
確率分布を出力するモデルを考えてみる。  
確率とlogitの両方を返すモデルを作成する。  
KerasModelとTorchModelでは「output types」のリストを指定することが可能。特定の出力がpredictionを持っている場合, prediction()を呼び出した時に返されるべき通常の出力であることを意味する。lossを持っている場合、出力の代わりに損失関数に渡されることを意味する。

In [13]:
class ClassificationModel(tf.keras.Model):

    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(1000, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1)

    def call(self, inputs, training=False):
        y = self.dense1(inputs)
        if training:
            y = tf.nn.dropout(y, 0.5)
        logits = self.dense2(y)
        output = tf.nn.sigmoid(logits)
        return output, logits

keras_model = ClassificationModel()
output_types = ['prediction', 'loss']
model = dc.models.KerasModel(keras_model, dc.models.losses.SigmoidCrossEntropy(), output_types=output_types)

In [14]:
# baceデータの読み込み
tasks, datasets, transformers = dc.molnet.load_bace_classification(featurizer='ECFP', split='scaffold')
train, valid, test = datasets
model.fit(train, nb_epoch=100)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
print('training set score: {}'.format(model.evaluate(train, [metric])))
print('test set score: {}'.format(model.evaluate(test, [metric])))

'split' is deprecated.  Use 'splitter' instead.


training set score: {'roc_auc_score': 0.9996116504854369}
test set score: {'roc_auc_score': 0.7567934782608696}


* トレーニング中のチェックポイントを自動的に保存。
* Logging progress to the console, to TensorBoard, or to Weights & Biases.
* f(outputs, labels, weights)形式の関数で定義したカスタム損失関数
* ValidationCallbackクラスを使用した早期停止。
* 事前に訓練されたモデルからのパラメータのロード
* モデル出力の不確実性の推定
* サリエンシーマッピングによる重要な特徴の識別
    - reference; https://keisen.github.io/keras-vis-docs-ja/visualizations/saliency/#_1
    - example: https://github.com/raghakot/keras-vis/tree/master/examples